In [1]:
using DynamicHMCModels

ProjDir = rel_path_d("..", "scripts", "12")

df = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(df) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
df[:logpop] = map((x) -> log(x), df[:population]);
df[:society] = 1:10;

first(df[[:total_tools, :logpop, :society]], 5)

struct m_12_06d_model{TY <: AbstractVector, TX <: AbstractMatrix,
  TS <: AbstractVector}
    "Observations (total_tools)."
    y::TY
    "Covariates (logpop)"
    X::TX
    "Society"
    S::TS
    "Number of observations (10)"
    N::Int
    "Number of societies (also 10)"
    N_societies::Int
end

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::m_12_06d_model)(θ)
    @unpack y, X, S, N, N_societies = problem   # extract the data
    @unpack β, α, σ = θ  # β : a, bp, α : a_society
    ll = 0.0
    ll += logpdf(Cauchy(0, 1), σ)
    ll += sum(logpdf.(Normal(0, σ), α)) # α[1:10]
    ll += sum(logpdf.(Normal(0, 10), β[1])) # a
    ll += sum(logpdf.(Normal(0, 1), β[2])) # a
    ll += sum(
      [loglikelihood(Poisson(exp(α[S[i]] + dot(X[i, :], β))), [y[i]]) for i in 1:N]
    )
    ll
end

Instantiate the model with data and inits.

In [4]:
N = size(df, 1)
N_societies = length(unique(df[:society]))
X = hcat(ones(Int64, N), df[:logpop]);
S = df[:society]
y = df[:total_tools]
p = m_12_06d_model(y, X, S, N, N_societies);
θ = (β = [1.0, 0.25], α = rand(Normal(0, 1), N_societies), σ = 0.2)
p(θ)

-343.886751153015

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::m_12_06d_model) =
    as( (β = as(Array, size(p.X, 2)), α = as(Array, p.N_societies), σ = asℝ₊) )
# Wrap the problem with a transformation, then use Flux for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));
#∇P = ADgradient(:ForwardDiff, P);

Tune and sample.

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.0018 s/step ...done
MCMC, adapting ϵ (25 steps)
0.0027 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0026 s/step ...done
MCMC, adapting ϵ (100 steps)
0.0013 s/step ...done
MCMC, adapting ϵ (200 steps)
0.00063 s/step ...done
MCMC, adapting ϵ (400 steps)
0.00044 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0004 s/step ...done
MCMC (1000 steps)
0.00041 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :α, :σ),Tuple{Array{Float64,1},Array{Float64,1},Float64}},1}:
 (β = [0.67241, 0.301183], α = [-0.183229, 0.270681, -0.0307498, 0.386632, 0.248597, -0.25905, -0.010782, -0.328266, 0.314811, -0.0891361], σ = 0.38494921573548035)   
 (β = [1.51479, 0.209675], α = [-0.125667, -0.0430555, -0.0658977, 0.370459, 0.188604, -0.159673, 0.41777, -0.368149, 0.361182, 0.11805], σ = 0.20972882074553492)     
 (β = [1.56114, 0.205465], α = [-0.0849418, 0.0127862, -0.047982, 0.369611, 0.255562, -0.154154, 0.509147, -0.338476, 0.41561, 0.2411], σ = 0.19538527224829222)       
 (β = [0.462974, 0.316918], α = [-0.139035, 0.0556719, 0.0812954, 0.316954, 0.288384, -0.247805, 0.391874, -0.00454889, 0.389447, -0.00967732], σ = 0.4513480493345123)
 (β = [1.79262, 0.180792], α = [-0.242727, -0.257559, -0.045624, 0.10806, 0.164431, 0.0401714, 0.409299, -0.200425, 0.291961, 0.187428], σ = 0.1908141898612518)       

Extract the parameter posterior means.

In [8]:
posterior_β = mean(posterior[i].β for i in 1:length(posterior))
posterior_α = mean(posterior[i].α for i in 1:length(posterior))
posterior_σ = mean(posterior[i].σ for i in 1:length(posterior))

0.3047503112565217

Effective sample sizes (of untransformed draws)

In [9]:
ess = mapslices(effective_sample_size, get_position_matrix(chain); dims = 1)
ess

1×13 Array{Float64,2}:
 646.381  727.478  776.575  1000.0  …  996.967  634.171  624.81  449.76

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.94, min/25%/median/75%/max: 0.53 0.92 0.97 0.99 1.0
  termination: AdjacentTurn => 9% DoubledTurn => 91%
  depth: 3 => 4% 4 => 94% 5 => 2% 6 => 0%


CmdStan result

In [11]:
m_12_6_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                            Mean                SD               Naive SE             MCSE            ESS
            a          1.076167468  0.7704872560 0.01218247319 0.0210530022 1000.000000
           bp         0.263056273  0.0823415805 0.00130193470 0.0022645077 1000.000000
  a_society.1   -0.191723568  0.2421382537 0.00382854195 0.0060563054 1000.000000
  a_society.2    0.054569029  0.2278506876 0.00360263570 0.0051693148 1000.000000
  a_society.3   -0.035935050  0.1926364647 0.00304584994 0.0039948433 1000.000000
  a_society.4    0.334355037  0.1929971201 0.00305155241 0.0063871707  913.029080
  a_society.5    0.049747513  0.1801287716 0.00284808595 0.0043631095 1000.000000
  a_society.6   -0.311903245  0.2096126337 0.00331426674 0.0053000536 1000.000000
  a_society.7    0.148637507  0.1744680594 0.00275858223 0.0047660246 1000.000000
  a_society.8   -0.164567976  0.1821341074 0.00287979309 0.0034297298 1000.000000
  a_society.9    0.277066965  0.1758237250 0.00278001719 0.0055844175  991.286501
 a_society.10   -0.094149204  0.2846206232 0.00450024719 0.0080735022 1000.000000
sigma_society    0.310352849  0.1374834682 0.00217380450 0.0057325226  575.187461
";

Show means

In [12]:
[posterior_β, posterior_α, posterior_σ]

3-element Array{Any,1}:
  [1.1003, 0.261396]                                                                                          
  [-0.190581, 0.0412796, -0.046996, 0.326682, 0.0389228, -0.321317, 0.145857, -0.177955, 0.273507, -0.0964698]
 0.3047503112565217                                                                                           

End of m12.6d.jl

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*